In [3]:
from backtester import Backtest
from backtester.strategy import Strategy, StrategyLeg
from backtester.enums import Type, Direction, Stock
from backtester.datahandler import HistoricalOptionsData, TiingoData
# Cleaned up data
options_data = HistoricalOptionsData(
    "/Users/lambdaclass/options_backtester_copy/backtester/examples/options_data_clean_v2.h5",
    key="/SPX",
    where='quotedate >= "2014-12-15" & quotedate <= "2015-03-15"')

options_data._data = options_data._data[(options_data._data['optionroot'] == "SPX160617P00650000")
                                        | (options_data._data['optionroot'] == "SPX160617C00650000")
                                        | (options_data._data['optionroot'] == "SPX160617P00700000")
                                        | (options_data._data['optionroot'] == "SPX160617C00700000")]

schema = options_data.schema
options_data._data.at[5857589, 'ask'] = 100  #SPX6500 call 2014-12-15
options_data._data.at[5857589, 'bid'] = 0.5  #SPX6500 call  2014-12-15
options_data._data.at[5857590, 'ask'] = 10  #SPX7000 call  2014-12-15
options_data._data.at[5857590, 'bid'] = 0.5  #SPX7000 call  2014-12-15
options_data._data.at[6045133, 'ask'] = 1  #SPX6500 put 2015-02-02
options_data._data.at[6045133, 'bid'] = 0.5

options_data._data.at[6045059, 'bid'] = 1  #SPX6500 call 2015-02-02
options_data._data.at[6045060, 'ask'] = 1

options_data._data.at[6151669, 'bid'] = 100  #SPX6500 call 2015-03-02
options_data._data.at[6151743, 'bid'] = 100
options_schema = options_data.schema
put_otm = Strategy(options_schema)

leg_1 = StrategyLeg("leg_1", options_schema, option_type=Type.PUT, direction=Direction.BUY)
leg_1.entry_filter = (options_schema.underlying == "SPX") & (options_schema.dte >= 60)
leg_1.exit_filter = (options_schema.dte <= 30)

leg_2 = StrategyLeg("leg_2", options_schema, option_type=Type.CALL, direction=Direction.BUY)
leg_2.entry_filter = (options_schema.underlying == "SPX") & (options_schema.dte >= 60)
leg_2.exit_filter = (options_schema.dte <= 30)

put_otm.add_legs([leg_1, leg_2])
options_data.columns = [
    'underlying', 'underlying_last', 'optionroot', 'type', 'expiration', 'quotedate', 'strike', 'last', 'bid', 'ask',
    'volume', 'openinterest', 'impliedvol', 'delta', 'gamma', 'theta', 'vega', 'optionalias', 'dte'
]
options_data.quotedate = options_data.quotedate.dt.tz_localize(None)
options_data._data.quotedate = options_data._data.quotedate.dt.tz_localize(None)

allocation = {'cash': 0, 'stocks': 97, 'options': 3}

bt = Backtest(allocation=allocation)

#asset_data = HistoricalAssetData('/Users/lambdaclass/options_backtester_copy/data/ivy_5assets.csv')
asset_data = TiingoData('/Users/lambdaclass/options_backtester_copy/data/ivy_5assets.csv')
asset_data._data.date = asset_data._data.date.dt.tz_localize(None)
asset_data.start_date = min(asset_data._data.date)
asset_data.end_date = max(asset_data._data.date)
VTI = Stock("VTI", 0.2)
VEU = Stock("VEU", 0.2)
BND = Stock("BND", 0.2)
VNQ = Stock("VNQ", 0.2)
DBC = Stock("DBC", 0.2)
bt.stocks = [VTI, VEU, BND, VNQ, DBC]
asset_data._data = asset_data._data.query('date >= "2014-12-15" & date <= "2015-03-15"')
bt.options_data = options_data
bt._options_strategy = put_otm
bt.stocks_data = asset_data
bt.stocks_data.start_date = min(bt.stocks_data['date'])
bt.stocks_data.end_date = max(bt.stocks_data['date'])

bt.current_cash = 1_000_000
bt.run(rebalance_freq=1)
bt.balance

# Exit thresholds: we exit if the loss or profit on the investment on an entry is greater than 25%.
#put_otm.exit_thresholds = (0.25, 0.25)

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


,total capital,cash,VTI,VEU,BND,VNQ,DBC,options qty,calls capital,puts capital,stocks qty,options capital,stocks capital,% change,accumulated return
2014-12-14,1000000.000000,1000000.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02,964109.713884,-20071.346493,201294.434010,199500.085077,194650.276734,202603.740636,186132.523921,2.0,0.0,80.0,23291.0,80.0,984181.060377,-0.035890,0.964110
2015-01-05,955268.364042,-20071.346493,197835.636144,195098.090912,195215.504398,203712.615403,183477.863678,2.0,0.0,150.0,23291.0,150.0,975339.710535,-0.009170,0.955268
2015-01-06,952427.771297,-20071.346493,195897.188989,193345.840807,195780.732062,205733.231644,181742.124289,2.0,0.0,160.0,23291.0,160.0,972499.117790,-0.002974,0.952428
2015-01-07,959050.963830,-20071.346493,198234.728206,195482.731179,195898.487825,208887.364312,180618.998802,2.0,0.0,130.0,23291.0,130.0,979122.310323,0.006954,0.959051
2015-01-08,966480.074540,-20071.346493,201731.534839,198217.950854,195592.322840,209675.897479,181333.715021,2.0,0.0,330.0,23291.0,330.0,986551.421034,0.007746,0.966480
2015-01-09,963818.481599,-20071.346493,200059.149058,197106.767861,195922.038978,209774.464125,181027.408070,2.0,0.0,330.0,23291.0,330.0,983889.828092,-0.002754,0.963818
2015-01-12,959459.914778,-20071.346493,198633.820267,196294.749519,196181.101657,211376.172121,177045.417706,2.0,0.0,330.0,23291.0,330.0,979531.261271,-0.004522,0.959460
2015-01-13,959225.606153,-20071.346493,198177.715054,197363.194705,196369.510878,211055.830522,176330.701487,2.0,0.0,150.0,23291.0,150.0,979296.952646,-0.000244,0.959226
2015-01-14,961583.974613,-20071.346493,197094.465173,196593.914172,196816.982779,212879.313471,178270.645511,2.0,0.0,150.0,23291.0,150.0,981655.321106,0.002459,0.961584


In [3]:
bt.trade_log

leg_1                                                      \
             contract underlying expiration type strike   cost      order   
0  SPX160617P00650000        SPX 2016-06-17  put  650.0  390.0  Order.BTO   
1  SPX160617P00700000        SPX 2016-06-17  put  700.0  350.0  Order.BTO   

                leg_2                                                         \
             contract underlying expiration  type strike     cost      order   
0  SPX160617C00650000        SPX 2016-06-17  call  650.0  10000.0  Order.BTO   
1  SPX160617C00700000        SPX 2016-06-17  call  700.0    100.0  Order.BTO   

    totals                   
      cost   qty       date  
0  10390.0   2.0 2014-12-15  
1    450.0  64.0 2015-02-02